In [1]:
import pandas as pd
df = pd.read_excel('/content/themes_csat.xlsx')
print(df.head())

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


   Respondent Id       Completed Date  CSAT score  \
0       11160013  2025-06-05 13:55:09           7   
1       11160003  2025-06-05 12:02:55           8   
2       11159949  2025-06-05 09:27:57           3   
3       11159948  2025-06-05 08:13:12           7   
4       11160137  2025-07-05 21:02:13          10   

  What are your most important reasons for giving us that score?  CONTRACT_ID  \
0  Generally your staff are good but I am horrifi...              TD035785047   
1  Prompt and helful response to requests for ass...                314601364   
2  Inefficient inhuman ID process.  Slow supply t...                450919618   
3                                                NaN              TD782511638   
4                                                NaN                237485095   

               Sub-Product  \
0                      NaN   
1        AMP SAVER ACCOUNT   
2   OFFSET DEPOSIT ACCOUNT   
3                      NaN   
4  AMP ESSENTIAL HOME LOAN   

   How would 

In [4]:
# ===============================================================
# TRAIN DISTILBERT MODEL FOR CSAT THEMES (ONLY TRAINING SET)
# FILTER: CSAT Score < 6
# ===============================================================

import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# -----------------------------------------------------------
# 1. Load Data
# -----------------------------------------------------------
#file_path = "themes_csat.xlsx"
#df = pd.read_excel(file_path)

DESC_COL = "What are your most important reasons for giving us that score?"
THEME_COL = "CSAT Theme"
SCORE_COL = "CSAT score"

data = df[[DESC_COL, THEME_COL, SCORE_COL]].copy()

# Convert descriptions to string
data[DESC_COL] = data[DESC_COL].astype(str)

# -----------------------------------------------------------
# 2. Clean Themes (0/null → Others)
# -----------------------------------------------------------
data[THEME_COL] = data[THEME_COL].astype("object")
data[THEME_COL] = data[THEME_COL].replace([0, "0"], np.nan)
data[THEME_COL] = data[THEME_COL].astype(str).str.strip()
data[THEME_COL] = data[THEME_COL].replace(["", "nan", "NaN", "None"], np.nan)
data["CSAT Theme Cleaned"] = data[THEME_COL].fillna("Others")

# -----------------------------------------------------------
# 3. Filter: CSAT Score < 6
# -----------------------------------------------------------
filtered = data[data[SCORE_COL] < 6].copy()

print("Rows with CSAT Score < 6:", filtered.shape[0])

# -----------------------------------------------------------
# 4. Prepare Labels for Model
# -----------------------------------------------------------
label_encoder = LabelEncoder()
filtered["label"] = label_encoder.fit_transform(filtered["CSAT Theme Cleaned"])

num_labels = len(label_encoder.classes_)
print("Number of unique themes:", num_labels)

# -----------------------------------------------------------
# 5. Prepare HuggingFace Dataset
# -----------------------------------------------------------
dataset = Dataset.from_pandas(
    filtered[[DESC_COL, "label"]].rename(columns={DESC_COL: "text"})
)

# -----------------------------------------------------------
# 6. Load Tokenizer & Tokenize Text
# -----------------------------------------------------------
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)

# -----------------------------------------------------------
# 7. Load DistilBERT Model
# -----------------------------------------------------------
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

# -----------------------------------------------------------
# 8. Training Arguments
# -----------------------------------------------------------
training_args = TrainingArguments(
    output_dir="./csat_distilbert",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=200,
    evaluation_strategy="no",   # NO TESTING — ONLY TRAINING
    logging_dir="./logs",
)

# -----------------------------------------------------------
# 9. Trainer (ONLY TRAINING)
# -----------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("\nTraining started...")
trainer.train()
print("\nTraining completed.")

# -----------------------------------------------------------
# 10. Predict themes for the same dataset
# -----------------------------------------------------------
predictions = trainer.predict(tokenized_dataset).predictions
predicted_labels = predictions.argmax(axis=1)

filtered["Predicted Theme"] = label_encoder.inverse_transform(predicted_labels)

print("\nSample predictions:")
print(filtered[[DESC_COL, "CSAT Theme Cleaned", "Predicted Theme"]].head())

# -----------------------------------------------------------
# 11. Save Output
# -----------------------------------------------------------
output_path = "csat_filtered_distilbert_predictions.xlsx"
filtered.to_excel(output_path, index=False)

print(f"\nSaved predictions to {output_path}")


Rows with CSAT Score < 6: 418
Number of unique themes: 22


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/418 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [5]:
# ===============================================================
# TRAIN DISTILBERT MODEL FOR CSAT THEMES (ONLY TRAINING SET)
# FILTER: CSAT Score < 6
# ===============================================================

import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# -----------------------------------------------------------
# 1. Load Data
# -----------------------------------------------------------
# If df is already in memory, this is fine. If not, uncomment below:
# file_path = "themes_csat.xlsx"
# df = pd.read_excel(file_path)

DESC_COL = "What are your most important reasons for giving us that score?"
THEME_COL = "CSAT Theme"
SCORE_COL = "CSAT score"   # Make sure this matches the exact column name in df

# Keep only the needed columns
data = df[[DESC_COL, THEME_COL, SCORE_COL]].copy()

# Convert descriptions to string
data[DESC_COL] = data[DESC_COL].astype(str)

# -----------------------------------------------------------
# 2. Clean Themes (0/null → Others)
# -----------------------------------------------------------
data[THEME_COL] = data[THEME_COL].astype("object")
data[THEME_COL] = data[THEME_COL].replace([0, "0"], np.nan)
data[THEME_COL] = data[THEME_COL].astype(str).str.strip()
data[THEME_COL] = data[THEME_COL].replace(["", "nan", "NaN", "None"], np.nan)
data["CSAT Theme Cleaned"] = data[THEME_COL].fillna("Others")

# -----------------------------------------------------------
# 3. Filter: CSAT Score < 6
# -----------------------------------------------------------
# Ensure score is numeric
data[SCORE_COL] = pd.to_numeric(data[SCORE_COL], errors="coerce")

filtered = data[data[SCORE_COL] < 6].copy()
filtered = filtered[filtered[DESC_COL].str.strip() != ""]  # drop empty text

print("Rows with CSAT Score < 6:", filtered.shape[0])

# If nothing to train on, stop here
if filtered.shape[0] == 0:
    raise ValueError("No rows with CSAT Score < 6 after filtering.")

# -----------------------------------------------------------
# 4. Prepare Labels for Model
# -----------------------------------------------------------
label_encoder = LabelEncoder()
filtered["label"] = label_encoder.fit_transform(filtered["CSAT Theme Cleaned"])

num_labels = len(label_encoder.classes_)
print("Number of unique themes:", num_labels)
print("Themes:", list(label_encoder.classes_))

# -----------------------------------------------------------
# 5. Prepare HuggingFace Dataset
# -----------------------------------------------------------
dataset = Dataset.from_pandas(
    filtered[[DESC_COL, "label"]].rename(columns={DESC_COL: "text"})
)

# -----------------------------------------------------------
# 6. Load Tokenizer & Tokenize Text
# -----------------------------------------------------------
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize, batched=True)

# HuggingFace Trainer expects these columns for inputs
tokenized_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

# -----------------------------------------------------------
# 7. Load DistilBERT Model
# -----------------------------------------------------------
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

# -----------------------------------------------------------
# 8. Training Arguments  (NO evaluation_strategy here)
# -----------------------------------------------------------
training_args = TrainingArguments(
    output_dir="./csat_distilbert",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=200,
    logging_dir="./logs",
    # remove evaluation_strategy for older transformers versions
)

# -----------------------------------------------------------
# 9. Trainer (ONLY TRAINING)
# -----------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("\nTraining started...")
trainer.train()
print("\nTraining completed.")

# -----------------------------------------------------------
# 10. Predict themes for the same dataset (training set)
# -----------------------------------------------------------
raw_preds = trainer.predict(tokenized_dataset).predictions
predicted_label_ids = raw_preds.argmax(axis=1)
filtered["Predicted Theme"] = label_encoder.inverse_transform(predicted_label_ids)

print("\nSample predictions:")
print(filtered[[DESC_COL, "CSAT Theme Cleaned", "Predicted Theme"]].head())

# -----------------------------------------------------------
# 11. Save Output
# -----------------------------------------------------------
output_path = "csat_filtered_distilbert_predictions.xlsx"
filtered.to_excel(output_path, index=False)

print(f"\nSaved predictions to {output_path}")


Rows with CSAT Score < 6: 418
Number of unique themes: 22
Themes: ['Card management', 'Complaint process', 'Deposit application process', 'Deposit rates', 'Generic', 'Home lending process', 'Home lending rates', 'Identification requirements', 'Live chat', 'No CSAT Verbatim', 'Others', 'Payments and Transfers', 'Product and Policy', 'Reputation', 'Retention', 'Service - BCC', 'Service - EDB', 'Service - Lending', 'Service - general', 'Technology', 'Term Deposit process', 'Unresolved']


Map:   0%|          | 0/418 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training started...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:



KeyboardInterrupt: 

In [6]:
# ===============================================================
# TF-IDF + NAIVE BAYES MODEL FOR CSAT THEMES (ONLY TRAINING SET)
# FILTER: CSAT Score < 6
# ===============================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# -----------------------------------------------------------
# 1. Load Data
# -----------------------------------------------------------
# If df is already loaded elsewhere, do not reload.
# Otherwise uncomment:
# df = pd.read_excel("themes_csat.xlsx")

DESC_COL = "What are your most important reasons for giving us that score?"
THEME_COL = "CSAT Theme"
SCORE_COL = "CSAT score"   # ensure correct column name

# Keep necessary columns
data = df[[DESC_COL, THEME_COL, SCORE_COL]].copy()

# Convert descriptions to string
data[DESC_COL] = data[DESC_COL].astype(str)

# -----------------------------------------------------------
# 2. Clean Themes (0/null → Others)
# -----------------------------------------------------------
data[THEME_COL] = data[THEME_COL].astype(object)
data[THEME_COL] = data[THEME_COL].replace([0, "0"], np.nan)
data[THEME_COL] = data[THEME_COL].astype(str).str.strip()
data[THEME_COL] = data[THEME_COL].replace(["", "nan", "NaN", "None"], np.nan)
data["CSAT Theme Cleaned"] = data[THEME_COL].fillna("Others")

# -----------------------------------------------------------
# 3. Filter: CSAT Score < 6
# -----------------------------------------------------------
data[SCORE_COL] = pd.to_numeric(data[SCORE_COL], errors="coerce")
filtered = data[data[SCORE_COL] < 6].copy()

# Remove empty comments
filtered = filtered[filtered[DESC_COL].str.strip() != ""]

print("Rows with CSAT Score < 6:", filtered.shape[0])

if filtered.shape[0] == 0:
    raise ValueError("No rows with CSAT Score < 6 after filtering!")

# -----------------------------------------------------------
# 4. Label Encoding
# -----------------------------------------------------------
label_encoder = LabelEncoder()
filtered["label"] = label_encoder.fit_transform(filtered["CSAT Theme Cleaned"])

num_labels = len(label_encoder.classes_)
print("Themes:", list(label_encoder.classes_))

# -----------------------------------------------------------
# 5. TF-IDF Vectorizer
# -----------------------------------------------------------
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),      # unigrams + bigrams
    min_df=2,                # ignore very rare terms
    max_df=0.95,             # ignore overly common terms
    stop_words="english"
)

X = tfidf.fit_transform(filtered[DESC_COL])
y = filtered["label"]

print("TF-IDF Shape:", X.shape)

# -----------------------------------------------------------
# 6. Train Naive Bayes Model (MultinomialNB)
# -----------------------------------------------------------
nb_model = MultinomialNB()
nb_model.fit(X, y)

print("\nModel training completed.")

# -----------------------------------------------------------
# 7. Predict Themes on SAME Dataset (since no test split)
# -----------------------------------------------------------
predicted_labels = nb_model.predict(X)
filtered["Predicted Theme"] = label_encoder.inverse_transform(predicted_labels)

print("\nSample Predictions:")
print(filtered[[DESC_COL, "CSAT Theme Cleaned", "Predicted Theme"]].head())

# -----------------------------------------------------------
# 8. Save Output
# -----------------------------------------------------------
output_path = "csat_filtered_tfidf_nb_predictions.xlsx"
filtered.to_excel(output_path, index=False)

print(f"\nSaved predictions to {output_path}")


Rows with CSAT Score < 6: 418
Themes: ['Card management', 'Complaint process', 'Deposit application process', 'Deposit rates', 'Generic', 'Home lending process', 'Home lending rates', 'Identification requirements', 'Live chat', 'No CSAT Verbatim', 'Others', 'Payments and Transfers', 'Product and Policy', 'Reputation', 'Retention', 'Service - BCC', 'Service - EDB', 'Service - Lending', 'Service - general', 'Technology', 'Term Deposit process', 'Unresolved']
TF-IDF Shape: (418, 1214)

Model training completed.

Sample Predictions:
   What are your most important reasons for giving us that score?  \
2   Inefficient inhuman ID process.  Slow supply t...               
6   Since you introduced the new rule that the acc...               
7   Since you have been outsourcing to the Phillip...               
9   Very bad service. Considering leaving. But tha...               
12  My issue was not addressed from the complaints...               

   CSAT Theme Cleaned Predicted Theme  
2   Servic

In [7]:
# ===============================================================
# SEMANTIC MODEL CLASSIFICATION (SBERT + LOGISTIC REGRESSION)
# FILTER: CSAT Score < 6
# NO API KEY REQUIRED
# ===============================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report

# -----------------------------------------------------------
# 1. Load Data
# -----------------------------------------------------------
DESC_COL = "What are your most important reasons for giving us that score?"
THEME_COL = "CSAT Theme"
SCORE_COL = "CSAT score"

data = df[[DESC_COL, THEME_COL, SCORE_COL]].copy()
data[DESC_COL] = data[DESC_COL].astype(str)

# -----------------------------------------------------------
# 2. Clean Themes (0/null → Others)
# -----------------------------------------------------------
data[THEME_COL] = data[THEME_COL].astype(object)
data[THEME_COL] = data[THEME_COL].replace([0, "0"], np.nan)
data[THEME_COL] = data[THEME_COL].astype(str).str.strip()
data[THEME_COL] = data[THEME_COL].replace(["", "nan", "NaN", "None"], np.nan)
data["CSAT Theme Cleaned"] = data[THEME_COL].fillna("Others")

# -----------------------------------------------------------
# 3. Filter CSAT Score < 6
# -----------------------------------------------------------
data[SCORE_COL] = pd.to_numeric(data[SCORE_COL], errors="coerce")
filtered = data[data[SCORE_COL] < 6].copy()
filtered = filtered[filtered[DESC_COL].str.strip() != ""]

print("Rows with CSAT Score < 6:", filtered.shape[0])

# -----------------------------------------------------------
# 4. Label Encoding
# -----------------------------------------------------------
label_encoder = LabelEncoder()
filtered["label"] = label_encoder.fit_transform(filtered["CSAT Theme Cleaned"])

num_labels = len(label_encoder.classes_)
print("Themes:", list(label_encoder.classes_))

# -----------------------------------------------------------
# 5. Load Semantic Model (NO API KEY REQUIRED)
# -----------------------------------------------------------
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# -----------------------------------------------------------
# 6. Convert Text to Embeddings
# -----------------------------------------------------------
X_embeddings = model.encode(
    filtered[DESC_COL].tolist(),
    batch_size=32,
    show_progress_bar=True
)

y = filtered["label"]

print("Embedding shape:", X_embeddings.shape)

# -----------------------------------------------------------
# 7. Train Classifier on Semantic Embeddings
# -----------------------------------------------------------
clf = LogisticRegression(
    max_iter=3000,
    class_weight="balanced"
)

clf.fit(X_embeddings, y)

print("\nModel training completed.")

# -----------------------------------------------------------
# 8. Predict on SAME dataset (since no test split)
# -----------------------------------------------------------
predicted = clf.predict(X_embeddings)
filtered["Predicted Theme"] = label_encoder.inverse_transform(predicted)

print("\nSample predictions:")
print(filtered[[DESC_COL, "CSAT Theme Cleaned", "Predicted Theme"]].head())

# -----------------------------------------------------------
# 9. Save Results
# -----------------------------------------------------------
output_path = "csat_semantic_sbert_predictions.xlsx"
filtered.to_excel(output_path, index=False)

print(f"\nSaved predictions to {output_path}")


Rows with CSAT Score < 6: 418
Themes: ['Card management', 'Complaint process', 'Deposit application process', 'Deposit rates', 'Generic', 'Home lending process', 'Home lending rates', 'Identification requirements', 'Live chat', 'No CSAT Verbatim', 'Others', 'Payments and Transfers', 'Product and Policy', 'Reputation', 'Retention', 'Service - BCC', 'Service - EDB', 'Service - Lending', 'Service - general', 'Technology', 'Term Deposit process', 'Unresolved']


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Embedding shape: (418, 384)

Model training completed.

Sample predictions:
   What are your most important reasons for giving us that score?  \
2   Inefficient inhuman ID process.  Slow supply t...               
6   Since you introduced the new rule that the acc...               
7   Since you have been outsourcing to the Phillip...               
9   Very bad service. Considering leaving. But tha...               
12  My issue was not addressed from the complaints...               

   CSAT Theme Cleaned    Predicted Theme  
2   Service - Lending    Card management  
6       Service - EDB      Deposit rates  
7       Service - BCC      Service - BCC  
9       Service - BCC            Generic  
12  Service - general  Complaint process  

Saved predictions to csat_semantic_sbert_predictions.xlsx


In [8]:
# ===============================================================
# SBERT SEMANTIC MODEL – EVALUATION ON SAME DATA
# CONFUSION MATRIX + PRECISION + RECALL + F1
# ===============================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    confusion_matrix
)
from sentence_transformers import SentenceTransformer

# -----------------------------------------------------------
# 1. Load Data
# -----------------------------------------------------------
DESC_COL = "What are your most important reasons for giving us that score?"
THEME_COL = "CSAT Theme"
SCORE_COL = "CSAT score"

data = df[[DESC_COL, THEME_COL, SCORE_COL]].copy()
data[DESC_COL] = data[DESC_COL].astype(str)

# -----------------------------------------------------------
# 2. Clean Themes
# -----------------------------------------------------------
data[THEME_COL] = data[THEME_COL].astype(object)
data[THEME_COL] = data[THEME_COL].replace([0, "0"], np.nan)
data[THEME_COL] = data[THEME_COL].astype(str).str.strip()
data[THEME_COL] = data[THEME_COL].replace(["", "nan", "NaN", "None"], np.nan)
data["CSAT Theme Cleaned"] = data[THEME_COL].fillna("Others")

# -----------------------------------------------------------
# 3. Filter CSAT Score < 6
# -----------------------------------------------------------
data[SCORE_COL] = pd.to_numeric(data[SCORE_COL], errors="coerce")
filtered = data[data[SCORE_COL] < 6].copy()
filtered = filtered[filtered[DESC_COL].str.strip() != ""]

print("Rows with CSAT Score < 6:", filtered.shape[0])

# -----------------------------------------------------------
# 4. Label Encoding
# -----------------------------------------------------------
label_encoder = LabelEncoder()
filtered["label"] = label_encoder.fit_transform(filtered["CSAT Theme Cleaned"])

y_true = filtered["label"].values
labels = label_encoder.classes_
print("Themes:", list(labels))

# -----------------------------------------------------------
# 5. Load SBERT (NO API KEY REQUIRED)
# -----------------------------------------------------------
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# -----------------------------------------------------------
# 6. Convert Text to Embeddings
# -----------------------------------------------------------
X_embeddings = model.encode(
    filtered[DESC_COL].tolist(),
    batch_size=32,
    show_progress_bar=True
)

print("Embedding shape:", X_embeddings.shape)

# -----------------------------------------------------------
# 7. Train Classifier on Entire Dataset
# -----------------------------------------------------------
clf = LogisticRegression(max_iter=4000, class_weight="balanced")
clf.fit(X_embeddings, y_true)

# -----------------------------------------------------------
# 8. Predict on SAME DATA
# -----------------------------------------------------------
y_pred = clf.predict(X_embeddings)
filtered["Predicted Theme"] = label_encoder.inverse_transform(y_pred)

# -----------------------------------------------------------
# 9. EVALUATION METRICS (ON SAME DATA)
# -----------------------------------------------------------

# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print("\n===============================")
print("ACCURACY =", round(accuracy, 4))
print("===============================\n")

# Classification Report
print("=== PRECISION • RECALL • F1 ===")
print(classification_report(y_true, y_pred, target_names=labels))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

print("\n=== CONFUSION MATRIX ===")
print(cm_df)

# -----------------------------------------------------------
# 10. Save Output
# -----------------------------------------------------------
output_path = "csat_semantic_sbert_predictions.xlsx"
filtered.to_excel(output_path, index=False)

print(f"\nSaved predictions with evaluation metrics to {output_path}")


Rows with CSAT Score < 6: 418
Themes: ['Card management', 'Complaint process', 'Deposit application process', 'Deposit rates', 'Generic', 'Home lending process', 'Home lending rates', 'Identification requirements', 'Live chat', 'No CSAT Verbatim', 'Others', 'Payments and Transfers', 'Product and Policy', 'Reputation', 'Retention', 'Service - BCC', 'Service - EDB', 'Service - Lending', 'Service - general', 'Technology', 'Term Deposit process', 'Unresolved']


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Embedding shape: (418, 384)

ACCURACY = 0.6435

=== PRECISION • RECALL • F1 ===
                             precision    recall  f1-score   support

            Card management       0.46      1.00      0.63         6
          Complaint process       0.40      1.00      0.57         2
Deposit application process       0.75      1.00      0.86         3
              Deposit rates       0.56      1.00      0.71        10
                    Generic       0.56      0.76      0.65        29
       Home lending process       0.40      1.00      0.57         6
         Home lending rates       0.47      1.00      0.64         7
Identification requirements       0.43      1.00      0.60         3
                  Live chat       1.00      1.00      1.00         1
           No CSAT Verbatim       1.00      1.00      1.00        30
                     Others       0.67      0.15      0.25        26
     Payments and Transfers       0.38      1.00      0.55         3
         Product and P

In [9]:
# ===============================================================
# OPTIMIZED SBERT CLASSIFICATION + FULL DATA OUTPUT
# ===============================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sentence_transformers import SentenceTransformer

# ==============================
# 1. Load Required Columns
# ==============================
DESC_COL = "What are your most important reasons for giving us that score?"
THEME_COL = "CSAT Theme"
SCORE_COL = "CSAT score"

df_work = df.copy()  # keep original safe
df_work[DESC_COL] = df_work[DESC_COL].astype(str)

# ==============================
# 2. Clean Themes
# ==============================
df_work[THEME_COL] = df_work[THEME_COL].astype(object)
df_work[THEME_COL] = df_work[THEME_COL].replace([0, "0"], np.nan)
df_work[THEME_COL] = df_work[THEME_COL].astype(str).str.strip()
df_work[THEME_COL] = df_work[THEME_COL].replace(["", "nan", "NaN", "None"], np.nan)
df_work["CSAT Theme Cleaned"] = df_work[THEME_COL].fillna("Others")

# ==============================
# 3. Filter for CSAT Score < 6
# ==============================
df_work[SCORE_COL] = pd.to_numeric(df_work[SCORE_COL], errors="coerce")
final_df = df_work[df_work[SCORE_COL] < 6].copy()
final_df = final_df[final_df[DESC_COL].str.strip() != ""]

print("Total rows used for training + evaluation:", final_df.shape[0])

# ==============================
# 4. Label Encoding
# ==============================
label_encoder = LabelEncoder()
final_df["Actual_Label"] = label_encoder.fit_transform(final_df["CSAT Theme Cleaned"])
labels = label_encoder.classes_

# ==============================
# 5. SBERT Embeddings
# ==============================
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

X = model.encode(
    final_df[DESC_COL].tolist(),
    batch_size=32,
    show_progress_bar=True
)

y = final_df["Actual_Label"].values

# ==============================
# 6. Train Classifier
# ==============================
clf = LogisticRegression(max_iter=4000, class_weight="balanced")
clf.fit(X, y)

# ==============================
# 7. Predictions
# ==============================
y_pred = clf.predict(X)

final_df["Predicted_Label"] = y_pred
final_df["Predicted Theme"] = label_encoder.inverse_transform(y_pred)

# ==============================
# 8. Metrics (On Same Data)
# ==============================
accuracy = accuracy_score(y, y_pred)
cls_report = classification_report(y, y_pred, target_names=labels)
cm = pd.DataFrame(confusion_matrix(y, y_pred), index=labels, columns=labels)

print("\n========= ACCURACY =========")
print(round(accuracy, 4))

print("\n===== CLASSIFICATION REPORT =====")
print(cls_report)

print("\n===== CONFUSION MATRIX =====")
print(cm)

# ==============================
# 9. Save Full Final Output File
# ==============================
output_path = "SBERT_CSAT_Theme_Predictions.xlsx"
final_df.to_excel(output_path, index=False)

print(f"\nFile saved successfully with ALL columns → {output_path}")


Total rows used for training + evaluation: 418


Batches:   0%|          | 0/14 [00:00<?, ?it/s]


========= ACCURACY =========
0.6435

===== CLASSIFICATION REPORT =====
                             precision    recall  f1-score   support

            Card management       0.46      1.00      0.63         6
          Complaint process       0.40      1.00      0.57         2
Deposit application process       0.75      1.00      0.86         3
              Deposit rates       0.56      1.00      0.71        10
                    Generic       0.56      0.76      0.65        29
       Home lending process       0.40      1.00      0.57         6
         Home lending rates       0.47      1.00      0.64         7
Identification requirements       0.43      1.00      0.60         3
                  Live chat       1.00      1.00      1.00         1
           No CSAT Verbatim       1.00      1.00      1.00        30
                     Others       0.67      0.15      0.25        26
     Payments and Transfers       0.38      1.00      0.55         3
         Product and Policy   